# QR-Zerlegung

In [1]:
import sympy as sp
import IPython.display as dp

from util.sympy.linalg import backwards_substitution, forwards_substitution, pivot


def magnitude(x: sp.Matrix) -> sp.Number:
    s = 0
    for i in range(len(x)):
        s += x[i] ** 2

    return sp.sqrt(s)


def gen_householder_matrix(a: sp.Matrix, i: int, precision: int = -1) -> sp.Matrix:
    ai = a[:, 0]
    ei = sp.eye(ai.rows)[:, 0]

    v = ai + sp.sign(ai[0]) * magnitude(ai) * ei
    u = (1 / magnitude(v)) * v

    if precision == -1:
        u = sp.simplify(u)
    else:
        u = u.evalf(precision)

    h = sp.eye(ai.rows) - 2 * (u @ sp.transpose(u))
    if precision == -1:
        h = sp.simplify(h)

    v_str = f'v_{i + 1} = {sp.latex(v)}, \\; |v_i| = {sp.latex(magnitude(v))} '
    if precision == -1:
        v_str += '= ' + sp.latex(sp.simplify(magnitude(v)))
    else:
        v_str += '\\approx ' + sp.latex(magnitude(v).evalf(precision))

    dp.display(dp.Math(f'A_{i + 1} = {sp.latex(a)} \\; \\rightarrow \\; a_i = {sp.latex(ai)}'))
    dp.display(dp.Math(v_str))
    dp.display(dp.Math(f'u_{i + 1} = {sp.latex(u)} \\; \\rightarrow \\; u_{i + 1}^T = {sp.latex(sp.transpose(u))}'))
    dp.display(dp.Math(
        f'H_{i + 1} = I_{len(ai)} - 2 u_{i + 1} u_{i + 1}^T = I_{len(ai)} - 2 \\cdot {sp.latex(u @ sp.transpose(u))} = {sp.latex(h)}'))

    return h


def expand_matrix(mat: sp.Matrix, n: int):
    offset = n - mat.rows

    res = sp.eye(n)
    res[offset:, offset:] = mat
    return res


def qr_decompose(a: sp.Matrix, precision: int = -1):
    n = a.rows

    r = a.copy()
    q = sp.eye(n)

    for i in range(n - 1):
        dp.display(dp.Markdown(f'Iteration {i + 1}'))

        hi = gen_householder_matrix(r[i:, i:], i, precision)
        qi = expand_matrix(hi, n)

        r = qi @ r
        q = q @ sp.transpose(qi)

        if precision == -1:
            r = sp.simplify(r)
            q = sp.simplify(q)
        else:
            # fill lower column with zeros to fix floating point errors
            r = sp.Matrix(r)
            r[(i + 1):, i] = sp.zeros(n - i - 1, 1)

        dp.display(dp.Math(f'Q_{i + 1} = {sp.latex(qi)} \\rightarrow Q = Q \\cdot Q_{i + 1}^T = {sp.latex(q)}, \; R = Q_{i + 1} \\cdot R = {sp.latex(r)}'))

    return q, r


def qr(a: sp.Matrix, b: sp.Matrix, precision: int = -1):
    dp.display(dp.Math(f'A = {sp.latex(a)}, \quad b = {sp.latex(b)}'))

    dp.display(dp.Markdown('## QR-Zerlegung'))
    q, r = qr_decompose(a, precision)

    dp.display(dp.Markdown('Resultat'))
    dp.display(dp.Math(f'Q = {sp.latex(q)}, \\quad R = {sp.latex(r)}'))

    y = sp.transpose(q) @ b

    dp.display(dp.Markdown('## Rückwärtseinsetzen'))
    x = backwards_substitution(r, y)
    dp.display(dp.Math('x = ' + sp.latex(x)))
    return sp.simplify(x)

In [2]:
a = sp.Matrix([
    [1, 2, -1],
    [4, -2, 6],
    [3, 1, 0],
])

b = sp.Matrix([9, -4, 9])

_ = qr(a, b)

<IPython.core.display.Math object>

## QR-Zerlegung

Iteration 1

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

## Rückwärtseinsetzen

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>